In [ ]:
from lib.utils import *
from lib.models import *
from tqdm import tqdm
from torch import nn

In [ ]:
window_size = 101
trainloader,devloader,test_idx = load_data_cv(foldi=0)
device = 'cuda'
model = MLP(window_size=window_size).to(device=device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())
params = sum([p.flatten().size()[0] for p in list(model.parameters())])
print("Params: ",params)

In [ ]:
loss_tr = []
loss_dev = []
pbar = tqdm(range(10))

for epoch in pbar:
    # train loop
    model.train()
    loss_tr_total = 0
    for (X_tr,y_tr) in trainloader:
        X_tr,y_tr = X_tr.to(device),y_tr.to(device)
        logits = model(X_tr)
        loss = criterion(logits,y_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_tr_total += loss.item()
    loss_tr.append(loss_tr_total/len(trainloader))

    # dev loop
    model.eval()
    loss_dev_total = 0
    for (X_dv,y_dv) in devloader:
        X_dv,y_dv = X_dv.to(device),y_dv.to(device)
        logits = model(X_dv)
        loss = criterion(logits,y_dv)
        loss_dev_total += loss.item()
    loss_dev.append(loss_dev_total/len(devloader))
plt.plot(loss_tr,label='train')
plt.plot(loss_dev,label='dev')
plt.legend()

In [ ]:
from sklearn.metrics import f1_score,recall_score,precision_score
f1i = []
recalli = []
precisioni = []
for idx in test_idx:
    X,y = load_and_window_nursing_list([idx])
    loss,y_true,y_pred = test_evaluation(DataLoader(TensorDataset(X,y),batch_size=32,shuffle=True),model,criterion,plot=True)
    f1i.append(f1_score(y_true=y_true,y_pred=y_pred.round(),average='macro'))
    recalli.append(recall_score(y_true=y_true,y_pred=y_pred.round(),average='macro'))
    precisioni.append(precision_score(y_true=y_true,y_pred=y_pred.round(),average='macro'))

In [ ]:
sns.kdeplot(recalli,bw_adjust=.4)
sns.rugplot(recalli)
print(torch.tensor(recalli).mean()) #.4958
print(torch.tensor(recalli).std()) #.0063

In [ ]:
sns.kdeplot(f1i,bw_adjust=.4)
sns.rugplot(f1i)
print(torch.tensor(f1i).mean()) #.4958
print(torch.tensor(f1i).std()) #.0063

In [ ]:
sns.kdeplot(precisioni,bw_adjust=.4)
sns.rugplot(precisioni)
print(torch.tensor(precisioni).mean()) #.4958
print(torch.tensor(precisioni).std()) #.0063

In [ ]:
from sklearn.metrics import f1_score,recall_score,precision_score
X,y = load_and_window_nursing_list(test_idx)
loss,y_true,y_pred = test_evaluation(DataLoader(TensorDataset(X,y),batch_size=32,shuffle=True),model,criterion,plot=True)
print(f1_score(y_true=y_true,y_pred=y_pred.round(),average='macro'))
print(recall_score(y_true=y_true,y_pred=y_pred.round(),average='macro'))
print(precision_score(y_true=y_true,y_pred=y_pred.round(),average='macro'))